    <span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

# Filters and combines existing clusters based on tools used, start and end dates, and organization
## Rebuilds all clusters (will need to run daily)

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

from nanoHUB.application import Application
application = Application.get_instance()


nanoHUB - Serving Students, Researchers & Instructors


In [2]:
import sys
 
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI


## Pull data from SF - this will be a post SF filter

In [3]:
##########################################################################################
## 1 - Pull all tool_usage_cluster__c information
##     i) ID
##     ii) Ending_Date__c
##     iii) Starting_Date__c
##     iv) Organization__c
##     v) Semester__c
## 2 - Pull all ContactToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Contact__c
## 3 - Pull all ToolToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Tool__c
## 4 - determine which tool_usage_cluster__c are suspicious 
##     i) first, group tool_usage_cluster__c by tool__c used
##     ii) check starting and ending dates - if they overlap, make a note of the tool_usage_clusters
## 5 - combining tool_usage_cluster__c 
##     i) use one of the existing Ids, mark the others for deletion
##     ii) when combining, pull the relevant Tool__c information and Contact__c information
##
##########################################################################################

In [4]:
# db_1 = DB2SalesforceAPI(sf_login_params)
db_1 = application.new_salesforce_engine()

Obtained Salesforce access token ...... True


In [5]:
tool_cluster_df = db_1.query_data('Select ID, Ending_Date__c, Starting_Date__c, Organization__c,Semester__c \
                        from tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000cyX2RAAU
{"id":"7505w00000cyX2RAAU","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:33:41.000+0000","systemModstamp":"2022-01-31T03:33:41.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000cyX2RAAU","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:33:41.000+0000","systemModstamp":"2022-01-31T03:33:42.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":4576,"retries":0,"totalProcessingTime":661}
[Success] Bulk job completed successfully.


In [6]:
display(tool_cluster_df.head(5))

,Ending_Date__c,Id,Organization__c,Semester__c,Starting_Date__c
0,2011-01-30,a0w5w000009Q7CuAAK,a0r5w00000V42iXAAR,Fall,2010-09-05
1,2018-11-30,a0w5w000009Q7CvAAK,a0r5w00000V42iyAAB,Fall,2018-11-29
2,2019-11-28,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,Fall,2019-11-04
3,2017-10-29,a0w5w000009Q7CxAAK,a0r5w00000V42cCAAR,Fall,2017-10-21
4,2018-01-21,a0w5w000009Q7CyAAK,a0r5w00000V42cCAAR,Spring,2018-01-15


In [7]:
contact_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Contact__c from ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000cyX2WAAU
{"id":"7505w00000cyX2WAAU","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:33:52.000+0000","systemModstamp":"2022-01-31T03:33:53.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000cyX2WAAU","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:33:52.000+0000","systemModstamp":"2022-01-31T03:33:59.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":101755,"retries":0,"totalProcessingTime":6352}
[Success] Bulk job completed successfully.


In [8]:
contact_in_cluster_df.shape

(101755, 4)

In [9]:
display(contact_in_cluster_df.head(5))

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JG1EAAW,a0x5w00000cMzV5AAK,gekco_454,a0w5w000009Q7m3AAC
1,0035w000034I5sjAAC,a0x5w00000cMzVAAA0,lalzatev_454,a0w5w000009Q7m3AAC
2,0035w000034IQMqAAO,a0x5w00000cN122AAC,mcclurez_943,a0w5w000009Q7twAAC
3,0035w000034JG1EAAW,a0x5w00000cN28DAAS,gekco_443,a0w5w000009Q7lsAAC
4,0035w000034JJ0aAAG,a0x5w00000cN327AAC,dkearney_238,a0w5w000009Q7iZAAS


In [10]:
tools_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Tool__c from ToolToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000cyX2SAAU
{"id":"7505w00000cyX2SAAU","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:34:07.000+0000","systemModstamp":"2022-01-31T03:34:07.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000cyX2SAAU","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:34:07.000+0000","systemModstamp":"2022-01-31T03:34:09.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6477,"retries":0,"totalProcessingTime":584}
[Success] Bulk job completed successfully.


In [11]:
display(tools_in_cluster_df.head(5))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000NuT38AAF,cntbands-ext_326,a0w5w000009Q7jzAAC,a0s5w00000k5MPqAAM
1,a0y5w00000NuT3xAAF,cntbands-ext_347,a0w5w000009Q7kKAAS,a0s5w00000k5MPqAAM
2,a0y5w00000NuT3yAAF,adac_347,a0w5w000009Q7kKAAS,a0s5w00000k5MT4AAM
3,a0y5w00000NuT8fAAF,rtdnegf_227,a0w5w000009Q7iOAAS,a0s5w00000k5MT3AAM
4,a0y5w00000NuT8gAAF,pntoy_227,a0w5w000009Q7iOAAS,a0s5w00000k5MR0AAM


### determining suspicious tool_usage_clusters__c

In [12]:
## overarching goal: determine the cluster IDs that need further investigation

# grouping the clusters by tools used (i.e., clusters by tool)
tools_dict = {j: [] for j in tools_in_cluster_df['Tool__c'].to_list()} #stores relevant indexes per org

for i,j in enumerate(tools_in_cluster_df['Tool__c'].to_list()):
    tools_dict[j].append(i)

unique_tools = list(tools_dict.keys())

In [13]:
from copy import deepcopy

In [14]:
## check start and end dates for overlaps
overlaps = ['init'] # del index 0 later

cluster_ids = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
tool_cluster_id_indexes = tool_cluster_df['Id'].to_list()

for i,ii in enumerate(unique_tools):
    for j in tools_dict[ii]: # this requires a sequential comparison
        ## tools dict contains the tools_in_cluster_df index
        temp_set = deepcopy(tools_dict[ii])
        del temp_set[temp_set.index(j)]
        
        # pull id
        current_index = tool_cluster_id_indexes.index(cluster_ids[j]) #integer index
        
        current_org = tool_cluster_df.iloc[current_index,2]
        current_sem = tool_cluster_df.iloc[current_index,3]
        current_start = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,-1],'%Y-%m-%d')
        current_stop = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,0],'%Y-%m-%d')
        
        # sequential compare
        for k in temp_set:
            temp_index = tool_cluster_id_indexes.index(cluster_ids[k]) #integer index
            
            temp_org = tool_cluster_df.iloc[temp_index,2]
            temp_sem = tool_cluster_df.iloc[temp_index,3]
            temp_start = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,-1],'%Y-%m-%d')
            temp_stop = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,0],'%Y-%m-%d')
        
            if temp_org == current_org and temp_sem == current_sem:
                # now check dates for overlap - if there is overlap, assume same cluster
                if temp_start > current_stop or temp_stop < current_start: #case 1 - () [], # case 2 - [] ()
                    # do nothing
                    filler = 0
                else:
                    # include the overlap as a tuple
                    overlaps.append([cluster_ids[j],cluster_ids[k]])


In [15]:
## filter the set of overlaps
if overlaps[0] == 'init':
    del overlaps[0]

for i,j in enumerate(overlaps):
    j.sort()
    overlaps[i] = j

overlaps.sort()

In [16]:
overlaps[:5]

[['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFhAAN']]

In [17]:
import itertools

In [18]:
overlaps2 = list(k for k,_ in itertools.groupby(overlaps))

In [19]:
overlaps2[:9]

[['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFhAAN'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFiAAN'],
 ['a0w5w000009Q7D9AAK', 'a0w5w00000A8OFkAAN'],
 ['a0w5w000009Q7D9AAK', 'a0w5w00000A8OFlAAN'],
 ['a0w5w000009Q7DFAA0', 'a0w5w00000A8OFnAAN'],
 ['a0w5w000009Q7DFAA0', 'a0w5w00000A8UFNAA3'],
 ['a0w5w000009Q7DMAA0', 'a0w5w00000A8OFoAAN']]

In [20]:
print(len(overlaps),len(overlaps2))

2886 1184


#### now that we have the set of overlapping clusters, we need to specify a main one

In [21]:
## main cluster characteristics
## Part A
## 1. combine overlaps together (i.e., three overlapping clusters combined into 1) 
## 2. decide the main cluster
## Part B
## 1. start and end dates realignment - i.e., should take the min of all the start dates and max of all the ends
## 2. combine contacts
## 3. create two DataFrames, one for upsert and the other for deletion

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [22]:
## Part A
# collapse overlaps2
overlaps3 = ['init']
main_clusters = []

running_temp_init = True

for i,j in enumerate(overlaps2):
    if running_temp_init == True:
        main_clusters.append(j[0])
        overlaps_holder = deepcopy(j)
        running_temp_init = False
    else:
        if j[0] in overlaps_holder and j[1] not in overlaps_holder:
            overlaps_holder.append(j[1])
        else: 
            if j[0] not in overlaps_holder and j[1] not in overlaps_holder: #neither is in overlaps_holder
                overlaps3.append(overlaps_holder)

                # recalc
                main_clusters.append(j[0])
                overlaps_holder = deepcopy(j)

# final iteration
overlaps3.append(overlaps_holder)

In [23]:
len(main_clusters)

633

In [24]:
if overlaps3[0] == 'init':
    del overlaps3[0]
    
print(len(overlaps3))

633


In [25]:
display(main_clusters[:5])
display(overlaps3[:5])

['a0w5w000009Q7CwAAK',
 'a0w5w000009Q7CxAAK',
 'a0w5w000009Q7CzAAK',
 'a0w5w000009Q7D9AAK',
 'a0w5w000009Q7DFAA0']

[['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFhAAN', 'a0w5w00000A8OFiAAN'],
 ['a0w5w000009Q7D9AAK', 'a0w5w00000A8OFkAAN', 'a0w5w00000A8OFlAAN'],
 ['a0w5w000009Q7DFAA0', 'a0w5w00000A8OFnAAN', 'a0w5w00000A8UFNAA3']]

### Major step - building upsert and deletion dataframes
### Build 3 upserts and 3 deletions [corresponds to tool_usage_cluster__c, ContactToolClusterAssociation__c, ToolToolClusterAssociation__c]

In [26]:
## Part B
### build the upsert dataframe first
upsert_df = pd.DataFrame(columns=['Id','Ending_Date__c','Starting_Date__c']) #Id of the main cluster
upsert_df['Id'] = deepcopy(main_clusters)

In [27]:
for i,ii in enumerate(main_clusters):
    main_c_index = tool_cluster_df['Id'].to_list()
    c_start_date = tool_cluster_df.iloc[main_c_index.index(ii),-1]    
    c_end_date = tool_cluster_df.iloc[main_c_index.index(ii),0]
        
    for j,jj in enumerate(overlaps3[i]):
        j_start_date = tool_cluster_df.iloc[main_c_index.index(jj),-1]
        j_end_date = tool_cluster_df.iloc[main_c_index.index(jj),0]
        
        if j_start_date < c_start_date:
            c_start_date = deepcopy(j_start_date)
        
        if j_end_date > c_end_date:
            c_end_date = deepcopy(j_end_date)
        
    upsert_df.iloc[i,1:] = [c_end_date,c_start_date]

In [28]:
display(upsert_df.head(5))
display(upsert_df.tail(5))

,Id,Ending_Date__c,Starting_Date__c
0,a0w5w000009Q7CwAAK,2019-11-28,2019-11-04
1,a0w5w000009Q7CxAAK,2017-12-16,2017-10-03
2,a0w5w000009Q7CzAAK,2018-04-19,2018-02-05
3,a0w5w000009Q7D9AAK,2007-04-23,2007-02-07
4,a0w5w000009Q7DFAA0,2008-12-18,2008-10-05


,Id,Ending_Date__c,Starting_Date__c
628,a0w5w00000AXc7vAAD,2021-04-07,2021-03-27
629,a0w5w00000AXc7zAAD,2021-04-11,2021-04-05
630,a0w5w00000AXc8AAAT,2021-04-22,2021-04-04
631,a0w5w00000AXc8aAAD,2021-04-09,2021-04-02
632,a0w5w00000AXc8pAAD,2021-04-03,2021-03-27


In [29]:
### buliding the delete dataframe - should be all Ids in overlaps that are not main clusters
deletion_df = pd.DataFrame(columns=['Id']) #Id of the main cluster

deletion_ids = []

for i,ii in enumerate(main_clusters):
    for j in overlaps3[i][1:]:
        deletion_ids.append(j)
deletion_df['Id'] = deletion_ids

In [30]:
display(deletion_df.head(3))

,Id
0,a0w5w00000A8WKZAA3
1,a0w5w00000A8OJTAA3
2,a0w5w00000A8OFhAAN


In [31]:
## summary up to this point
## upsert_df db.send_data(upsert_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

## deletion_df db.delete_data(deletion_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

In [32]:
## updating contact and cluster mapping
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JG1EAAW,a0x5w00000cMzV5AAK,gekco_454,a0w5w000009Q7m3AAC
1,0035w000034I5sjAAC,a0x5w00000cMzVAAA0,lalzatev_454,a0w5w000009Q7m3AAC


(101755, 4)


In [33]:
# NOTE: there are contacts that belong to non-overlapping clusters
contact_usage_clusters = contact_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
contacts = contact_in_cluster_df['Contact__c'].to_list()
change_indexes = []

## check if current cluster belongs to overlaps3 - first need to convert overlaps3 into a direct list
overlaps4 = list(itertools.chain.from_iterable(overlaps3))

In [34]:
print(len(overlaps4))

1661


In [35]:
## populate the contacts that need changing
for i,j in enumerate(contact_usage_clusters):
    if j in overlaps4:
        change_indexes.append(i)

In [36]:
print(len(change_indexes))

38562


In [37]:
## search for each index that needs changing
for i in change_indexes:
    if contact_usage_clusters[i] not in main_clusters:
        #print('testouter')
        
        # need to find its main_cluster
        found_main = False
        main_counter = 0
        while found_main == False:
            if contact_usage_clusters[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                contact_usage_clusters[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [38]:
contact_in_cluster_df['Tool_Usage_Cluster__c'] = contact_usage_clusters

In [39]:
## remove duplicates from contact_in_cluster_df
contact_in_cluster_df = contact_in_cluster_df.drop_duplicates()
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JG1EAAW,a0x5w00000cMzV5AAK,gekco_454,a0w5w000009Q7m3AAC
1,0035w000034I5sjAAC,a0x5w00000cMzVAAA0,lalzatev_454,a0w5w000009Q7m3AAC


(101755, 4)


In [40]:
## tools - take max, i.e. set all constituent clusters of a main cluster to the main cluster
## then remove duplicates
display(tools_in_cluster_df.head(2))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000NuT38AAF,cntbands-ext_326,a0w5w000009Q7jzAAC,a0s5w00000k5MPqAAM
1,a0y5w00000NuT3xAAF,cntbands-ext_347,a0w5w000009Q7kKAAS,a0s5w00000k5MPqAAM


In [41]:
tuc = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()

# first find those cluster ids that are clustered
tool_cluster_ids = []
for i,j in enumerate(tuc):
    if j in overlaps4:
        tool_cluster_ids.append(i)

In [42]:
print(len(tool_cluster_ids))

2105


In [43]:
for i in tool_cluster_ids:
    if tuc[i] not in main_clusters: #then need to find its corresponding main cluster
        found_main = False
        main_counter = 0
        
        while found_main == False:
            if tuc[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                tuc[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [44]:
tools_in_cluster_df['Tool_Usage_Cluster__c'] = tuc
display(tools_in_cluster_df.head(2))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000NuT38AAF,cntbands-ext_326,a0w5w000009Q7jzAAC,a0s5w00000k5MPqAAM
1,a0y5w00000NuT3xAAF,cntbands-ext_347,a0w5w000009Q7kKAAS,a0s5w00000k5MPqAAM


## Send to SF

In [45]:
db_upsert_del = application.new_salesforce_engine()

Obtained Salesforce access token ...... True


In [46]:
db_upsert_del.object_id = 'tool_usage_cluster__c'
db_upsert_del.external_id = 'Id'

In [47]:
db_upsert_del.send_data(upsert_df)

[Success] Bulk job creation successful. Job ID = 7505w00000cyX2lAAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyX2lAAE
[Success] Closing job successful. Job ID = 7505w00000cyX2lAAE


In [48]:
db_upsert_del.check_bulk_status()

{'id': '7505w00000cyX2lAAE',
 'operation': 'upsert',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:38:05.000+0000',
 'systemModstamp': '2022-01-31T03:38:06.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [49]:
from pprint import pprint
pprint(db_upsert_del.check_bulk_failed_results())

''


#### upload contact_in_cluster_df

In [50]:
db_contact_upsert = application.new_salesforce_engine()

Obtained Salesforce access token ...... True


In [51]:
## need to delete first, then upload
contact_in_cluster_df_del = pd.DataFrame(contact_in_cluster_df['Id'])
contact_in_cluster_df_up = contact_in_cluster_df.drop(columns='Id')

In [52]:
display(contact_in_cluster_df_del.head(2))
display(contact_in_cluster_df_up.head(2))

,Id
0,a0x5w00000cMzV5AAK
1,a0x5w00000cMzVAAA0


,Contact__c,Name,Tool_Usage_Cluster__c
0,0035w000034JG1EAAW,gekco_454,a0w5w000009Q7m3AAC
1,0035w000034I5sjAAC,lalzatev_454,a0w5w000009Q7m3AAC


In [53]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Id'

In [54]:
db_contact_upsert.delete_data(contact_in_cluster_df_del)

[Success] Bulk job creation successful. Job ID = 7505w00000cyWv2AAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyWv2AAE
[Success] Closing job successful. Job ID = 7505w00000cyWv2AAE


In [55]:
db_contact_upsert.check_bulk_status()

{'id': '7505w00000cyWv2AAE',
 'operation': 'delete',
 'object': 'ContactToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:38:07.000+0000',
 'systemModstamp': '2022-01-31T03:38:08.000+0000',
 'state': 'UploadComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [56]:
pprint(db_contact_upsert.check_bulk_failed_results())

''


In [57]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Name'

In [58]:
db_contact_upsert.send_data(contact_in_cluster_df_up)

[Success] Bulk job creation successful. Job ID = 7505w00000cyX2qAAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyX2qAAE
[Success] Closing job successful. Job ID = 7505w00000cyX2qAAE


In [59]:
db_contact_upsert.check_bulk_status()

{'id': '7505w00000cyX2qAAE',
 'operation': 'upsert',
 'object': 'ContactToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:38:09.000+0000',
 'systemModstamp': '2022-01-31T03:38:12.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Name',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [60]:
pprint(db_contact_upsert.check_bulk_failed_results())

''


#### tools_in_cluster_df

In [61]:
db_tool_upsert = application.new_salesforce_engine()

Obtained Salesforce access token ...... True


In [62]:
## need to delete first, then upload
db_tool_upsert_del = pd.DataFrame(tools_in_cluster_df['Id'])
db_tool_upsert_up = tools_in_cluster_df.drop(columns='Id')

In [63]:
display(db_tool_upsert_del.head(2))
display(db_tool_upsert_up.head(2))

,Id
0,a0y5w00000NuT38AAF
1,a0y5w00000NuT3xAAF


,Name,Tool_Usage_Cluster__c,Tool__c
0,cntbands-ext_326,a0w5w000009Q7jzAAC,a0s5w00000k5MPqAAM
1,cntbands-ext_347,a0w5w000009Q7kKAAS,a0s5w00000k5MPqAAM


In [64]:
db_tool_upsert.external_id = 'Id'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [65]:
db_tool_upsert.delete_data(db_tool_upsert_del)

[Success] Bulk job creation successful. Job ID = 7505w00000cyX2vAAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyX2vAAE
[Success] Closing job successful. Job ID = 7505w00000cyX2vAAE


In [66]:
db_tool_upsert.check_bulk_status()

{'id': '7505w00000cyX2vAAE',
 'operation': 'delete',
 'object': 'ToolToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:38:13.000+0000',
 'systemModstamp': '2022-01-31T03:38:14.000+0000',
 'state': 'UploadComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [67]:
pprint(db_tool_upsert.check_bulk_failed_results())

''


In [68]:
## now upload

In [69]:
db_tool_upsert.external_id = 'Name'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [70]:
db_tool_upsert.send_data(db_tool_upsert_up)

[Success] Bulk job creation successful. Job ID = 7505w00000cyX30AAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyX30AAE
[Success] Closing job successful. Job ID = 7505w00000cyX30AAE


In [71]:
db_tool_upsert.check_bulk_status()

{'id': '7505w00000cyX30AAE',
 'operation': 'upsert',
 'object': 'ToolToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:38:15.000+0000',
 'systemModstamp': '2022-01-31T03:38:16.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Name',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [72]:
pprint(db_tool_upsert.check_bulk_failed_results())

''


#### delete clustered tool cluster ids

In [73]:
db_del = application.new_salesforce_engine()

Obtained Salesforce access token ...... True


In [74]:
db_del.object_id = 'tool_usage_cluster__c'
db_del.external_id = 'id'

In [75]:
db_del.delete_data(deletion_df)

[Success] Bulk job creation successful. Job ID = 7505w00000cyX35AAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyX35AAE
[Success] Closing job successful. Job ID = 7505w00000cyX35AAE


In [76]:
display(deletion_df.head(2))

,Id
0,a0w5w00000A8WKZAA3
1,a0w5w00000A8OJTAA3


In [77]:
db_del.check_bulk_status()

{'id': '7505w00000cyX35AAE',
 'operation': 'delete',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:38:17.000+0000',
 'systemModstamp': '2022-01-31T03:38:17.000+0000',
 'state': 'InProgress',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [78]:
pprint(db_del.check_bulk_failed_results())

''


In [79]:
### eventually, want to update DB2 with this information
